## 1. team

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import requests
import json
import os
import datetime
import time

In [2]:
def oppo(date, ssn): 
    
    ### the 'team' parameter takes in an nba team's three letter abbreviation. 'GSW' for the golden state warriors, for example
    ### the 'ssn' parameter takes in an nba season (2013-14 at the earliest). '2015-16' for example
    ### the 'web' parameter takes in 'AST' or 'pass' (or other variations of the two). 
    ### the 'web' parameter exists because you can create networks in which the lines represent either assists or passes -- the parameter is used to tell which one of the two you'd like to do
    ### example of usage: pass_web('GSW','2016-17','AST')
    
#     if (web in ['ast','AST','assist','assists','assisting','ASSIST','ASSISTS']):
#         web = 'assist'
#     elif (web in ['pass','passes','passing','PASS','PASSES']):
#         web = 'pass'
#     else:
#         return print('Error: Third parameter requires "assist" or "pass" input')
    
    # set headers  
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Referer': 'https://www.nba.com/'}
    
    url = 'https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerOrTeam=Team&PlayerPosition=&PtMeasureType=Drives&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
    
    json = requests.get(url, headers=headers).json()
    data = json['resultSets'][0]['rowSet']
    columns = json['resultSets'][0]['headers']
    tms = pd.DataFrame.from_records(data, columns=columns)
    teamName_dict = dict(zip(tms['TEAM_ID'], tms['TEAM_ABBREVIATION']))
    teamId_dict = dict(zip(tms['TEAM_ID'], [0 for i in range(len(tms['TEAM_ID']))]))
    win_dict = dict(zip(tms['TEAM_ID'], tms['W']))
    schedule_list = []
    while 0 in teamId_dict.values():
        for key in teamId_dict:
            if teamId_dict[key] == 0:
                url = 'https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=' + str(key) + '&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerOrTeam=Team&PlayerPosition=&PtMeasureType=Drives&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
                json = requests.get(url, headers=headers).json()
                data = json['resultSets'][0]['rowSet']
                columns = json['resultSets'][0]['headers']
                oppo = pd.DataFrame.from_records(data, columns=columns)['TEAM_ID'][0]
                teamId_dict[key] = 1
                teamId_dict[oppo] = 1
                schedule_list.append({key: win_dict[key], oppo: win_dict[oppo]})
                break
    
    ### using the scraped team_id, find all players who accumulated at least 10 assists with that team in the inputted season
    if not os.path.exists('./team'):
        os.mkdir('./team')
    date_name = date.replace('/', '-')
    if not os.path.exists(os.path.join('team', date_name)):
        os.mkdir(os.path.join('team', date_name))
    print(date)
    for team_id in schedule_list:
        print(team_id.keys())
        team1 = list(team_id.keys())[0]
        team2 = list(team_id.keys())[1]
        time.sleep(1)
        url = 'https://stats.nba.com/stats/teamdashboardbyopponent?DateFrom=' + str(date) + '&DateTo=' + str(date) + '&GameSegment=&LastNGames=0&LeagueID=&Location=&MeasureType=Base&Month=0&OpponentTeamID=' + str(team2) + '&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&TeamID=' + str(team1) + '&VsConference=&VsDivision='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        team_df = pd.DataFrame.from_records(data, columns=columns)
        team_df.to_csv(os.path.join('team', date_name, teamName_dict[team1] + '.csv'))
        time.sleep(1)
        url = 'https://stats.nba.com/stats/teamdashboardbyopponent?DateFrom=' + str(date) + '&DateTo=' + str(date) + '&GameSegment=&LastNGames=0&LeagueID=&Location=&MeasureType=Base&Month=0&OpponentTeamID=' + str(team1) + '&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&TeamID=' + str(team2) + '&VsConference=&VsDivision='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        team_df = pd.DataFrame.from_records(data, columns=columns)
        team_df.to_csv(os.path.join('team', date_name, teamName_dict[team2] + '.csv'))
        
    return schedule_list

In [3]:
# test
# list_1 = pass_web('12/25/2016','2016-17','AST')

In [4]:
schedule_list = dict()
data_list = [d.strftime('%m/%d/%Y') for d in pd.date_range(start='10/18/2022', end='04/09/2023')]
for data in data_list:
    list_schedule = oppo(data,'2022-23')
    schedule_list[data] = list_schedule

10/18/2022
dict_keys([1610612738, 1610612755])
dict_keys([1610612744, 1610612747])
10/19/2022
dict_keys([1610612737, 1610612745])
dict_keys([1610612751, 1610612740])
dict_keys([1610612766, 1610612759])
dict_keys([1610612741, 1610612748])
dict_keys([1610612739, 1610612761])
dict_keys([1610612742, 1610612756])
dict_keys([1610612743, 1610612762])
dict_keys([1610612765, 1610612753])
dict_keys([1610612754, 1610612764])
dict_keys([1610612763, 1610612752])
dict_keys([1610612750, 1610612760])
dict_keys([1610612757, 1610612758])
10/20/2022
dict_keys([1610612746, 1610612747])
dict_keys([1610612749, 1610612755])
10/21/2022
dict_keys([1610612737, 1610612753])
dict_keys([1610612738, 1610612748])
dict_keys([1610612751, 1610612761])
dict_keys([1610612766, 1610612740])
dict_keys([1610612741, 1610612764])
dict_keys([1610612743, 1610612744])
dict_keys([1610612765, 1610612752])
dict_keys([1610612745, 1610612763])
dict_keys([1610612754, 1610612759])
dict_keys([1610612750, 1610612762])
dict_keys([161061275

dict_keys([1610612760, 1610612764])
11/17/2022
dict_keys([1610612751, 1610612757])
dict_keys([1610612765, 1610612746])
dict_keys([1610612758, 1610612759])
11/18/2022
dict_keys([1610612738, 1610612740])
dict_keys([1610612766, 1610612739])
dict_keys([1610612741, 1610612753])
dict_keys([1610612742, 1610612743])
dict_keys([1610612765, 1610612747])
dict_keys([1610612744, 1610612752])
dict_keys([1610612745, 1610612754])
dict_keys([1610612763, 1610612760])
dict_keys([1610612748, 1610612764])
dict_keys([1610612749, 1610612755])
dict_keys([1610612756, 1610612762])
11/19/2022
dict_keys([1610612737, 1610612761])
dict_keys([1610612754, 1610612753])
dict_keys([1610612746, 1610612759])
dict_keys([1610612750, 1610612755])
dict_keys([1610612757, 1610612762])
11/20/2022
dict_keys([1610612751, 1610612763])
dict_keys([1610612766, 1610612764])
dict_keys([1610612739, 1610612748])
dict_keys([1610612742, 1610612743])
dict_keys([1610612765, 1610612758])
dict_keys([1610612744, 1610612745])
dict_keys([161061274

dict_keys([1610612745, 1610612757])
dict_keys([1610612746, 1610612764])
dict_keys([1610612763, 1610612760])
dict_keys([1610612748, 1610612759])
dict_keys([1610612749, 1610612762])
dict_keys([1610612740, 1610612756])
12/18/2022
dict_keys([1610612738, 1610612753])
dict_keys([1610612751, 1610612765])
dict_keys([1610612766, 1610612743])
dict_keys([1610612741, 1610612750])
dict_keys([1610612744, 1610612761])
dict_keys([1610612754, 1610612752])
dict_keys([1610612747, 1610612764])
12/19/2022
dict_keys([1610612737, 1610612753])
dict_keys([1610612766, 1610612758])
dict_keys([1610612739, 1610612762])
dict_keys([1610612742, 1610612750])
dict_keys([1610612745, 1610612759])
dict_keys([1610612747, 1610612756])
dict_keys([1610612749, 1610612740])
dict_keys([1610612760, 1610612757])
dict_keys([1610612755, 1610612761])
12/20/2022
dict_keys([1610612741, 1610612748])
dict_keys([1610612743, 1610612763])
dict_keys([1610612765, 1610612762])
dict_keys([1610612744, 1610612752])
dict_keys([1610612756, 16106127

dict_keys([1610612744, 1610612764])
dict_keys([1610612745, 1610612747])
dict_keys([1610612754, 1610612749])
dict_keys([1610612763, 1610612756])
dict_keys([1610612750, 1610612762])
dict_keys([1610612752, 1610612761])
01/17/2023
dict_keys([1610612751, 1610612759])
dict_keys([1610612743, 1610612757])
dict_keys([1610612746, 1610612755])
dict_keys([1610612749, 1610612761])
01/18/2023
dict_keys([1610612737, 1610612742])
dict_keys([1610612766, 1610612745])
dict_keys([1610612739, 1610612763])
dict_keys([1610612743, 1610612750])
dict_keys([1610612754, 1610612760])
dict_keys([1610612746, 1610612762])
dict_keys([1610612747, 1610612758])
dict_keys([1610612748, 1610612740])
dict_keys([1610612752, 1610612764])
01/19/2023
dict_keys([1610612738, 1610612744])
dict_keys([1610612751, 1610612756])
dict_keys([1610612750, 1610612761])
dict_keys([1610612755, 1610612757])
01/20/2023
dict_keys([1610612737, 1610612752])
dict_keys([1610612751, 1610612762])
dict_keys([1610612739, 1610612744])
dict_keys([161061274

dict_keys([1610612745, 1610612760])
dict_keys([1610612747, 1610612740])
dict_keys([1610612763, 1610612762])
02/16/2023
dict_keys([1610612741, 1610612749])
dict_keys([1610612746, 1610612756])
dict_keys([1610612750, 1610612764])
02/17/2023
02/18/2023
02/19/2023
02/20/2023
02/21/2023
02/22/2023
02/23/2023
dict_keys([1610612738, 1610612754])
dict_keys([1610612739, 1610612743])
dict_keys([1610612742, 1610612759])
dict_keys([1610612765, 1610612753])
dict_keys([1610612744, 1610612747])
dict_keys([1610612763, 1610612755])
dict_keys([1610612740, 1610612761])
dict_keys([1610612760, 1610612762])
dict_keys([1610612757, 1610612758])
02/24/2023
dict_keys([1610612737, 1610612739])
dict_keys([1610612751, 1610612741])
dict_keys([1610612766, 1610612750])
dict_keys([1610612744, 1610612745])
dict_keys([1610612746, 1610612758])
dict_keys([1610612748, 1610612749])
dict_keys([1610612752, 1610612764])
dict_keys([1610612760, 1610612756])
02/25/2023
dict_keys([1610612738, 1610612755])
dict_keys([1610612766, 161

dict_keys([1610612752, 1610612753])
03/24/2023
dict_keys([1610612738, 1610612754])
dict_keys([1610612766, 1610612742])
dict_keys([1610612741, 1610612757])
dict_keys([1610612765, 1610612761])
dict_keys([1610612744, 1610612755])
dict_keys([1610612745, 1610612763])
dict_keys([1610612747, 1610612760])
dict_keys([1610612749, 1610612762])
dict_keys([1610612756, 1610612758])
dict_keys([1610612759, 1610612764])
03/25/2023
dict_keys([1610612737, 1610612754])
dict_keys([1610612751, 1610612748])
dict_keys([1610612743, 1610612749])
dict_keys([1610612746, 1610612740])
dict_keys([1610612755, 1610612756])
dict_keys([1610612758, 1610612762])
03/26/2023
dict_keys([1610612737, 1610612763])
dict_keys([1610612738, 1610612759])
dict_keys([1610612751, 1610612753])
dict_keys([1610612766, 1610612742])
dict_keys([1610612741, 1610612747])
dict_keys([1610612739, 1610612745])
dict_keys([1610612744, 1610612750])
dict_keys([1610612760, 1610612757])
dict_keys([1610612761, 1610612764])
03/27/2023
dict_keys([161061274

In [3]:
ssss = ['10/26/2021', '11/03/2021', '11/18/2021', '11/20/2021',
       '01/13/2022', '01/17/2022']
schedule_list = dict()
for sss in ssss:
    data_list = [d.strftime('%m/%d/%Y') for d in pd.date_range(start=sss, end=sss)]
    for data in data_list:
        print(data)
        list_schedule = oppo(data,'2021-22')
        schedule_list[data] = list_schedule

10/26/2021
dict_keys([1610612742, 1610612745])
dict_keys([1610612743, 1610612762])
dict_keys([1610612744, 1610612760])
dict_keys([1610612747, 1610612759])
dict_keys([1610612752, 1610612755])
11/03/2021
dict_keys([1610612737, 1610612751])
dict_keys([1610612738, 1610612753])
dict_keys([1610612766, 1610612744])
dict_keys([1610612741, 1610612755])
dict_keys([1610612739, 1610612757])
dict_keys([1610612742, 1610612759])
dict_keys([1610612743, 1610612763])
dict_keys([1610612754, 1610612752])
dict_keys([1610612746, 1610612750])
dict_keys([1610612740, 1610612758])
dict_keys([1610612761, 1610612764])
11/18/2021
dict_keys([1610612739, 1610612744])
dict_keys([1610612743, 1610612755])
dict_keys([1610612746, 1610612763])
dict_keys([1610612748, 1610612764])
dict_keys([1610612750, 1610612759])
dict_keys([1610612761, 1610612762])
11/20/2021
dict_keys([1610612737, 1610612766])
dict_keys([1610612738, 1610612760])
dict_keys([1610612745, 1610612752])
dict_keys([1610612754, 1610612740])
dict_keys([161061276

In [34]:
# print(schedule_list)
# import pickle
# pickle.dump(schedule_list, open( "schedule2.p", "wb" ))

## 2. player def

In [5]:
import pickle
import os
import time
import pandas as pd
import numpy as np
import requests
import json
s_list = pickle.load(open( "schedule4.p", "rb" ))

In [6]:
def oppo_player(date, ssn, schedule_list): 

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Referer': 'https://www.nba.com/'}

    if not os.path.exists('./def'):
        os.mkdir('./def')
    
    date_name = date.replace('/', '-')
    if not os.path.exists(os.path.join('def', date_name)):
        os.mkdir(os.path.join('def', date_name))
    match_list = schedule_list[date]
    for match in match_list:
        time.sleep(0.5)
        team1 = list(match.keys())[0]
        team2 = list(match.keys())[1]
        url = 'https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&DefenseCategory=Overall&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=&LeagueID=00&Location=&Month=&OpponentTeamID=&Outcome=&PORound=&PerMode=Totals&Period=&PlayerExperience=&PlayerID=&PlayerPosition=&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=' + str(team1) + '&VsConference=&VsDivision=&Weight='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        df = pd.DataFrame.from_records(data, columns=columns)
        df.to_csv(os.path.join('def', date_name, str(team1) + '.csv'))
        url = 'https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&DefenseCategory=Overall&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=&LeagueID=00&Location=&Month=&OpponentTeamID=&Outcome=&PORound=&PerMode=Totals&Period=&PlayerExperience=&PlayerID=&PlayerPosition=&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=' + str(team2) + '&VsConference=&VsDivision=&Weight='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        df = pd.DataFrame.from_records(data, columns=columns)
        df.to_csv(os.path.join('def', date_name, str(team2) + '.csv'))

In [7]:
data_list = [d.strftime('%m/%d/%Y') for d in pd.date_range(start='10/18/2022', end='04/09/2023')]
for date in data_list: 
    print(date)
    oppo_player(date, '2022-23', s_list)

10/18/2022
10/19/2022
10/20/2022
10/21/2022
10/22/2022
10/23/2022
10/24/2022
10/25/2022
10/26/2022
10/27/2022
10/28/2022
10/29/2022
10/30/2022
10/31/2022
11/01/2022
11/02/2022
11/03/2022
11/04/2022
11/05/2022
11/06/2022
11/07/2022
11/08/2022
11/09/2022
11/10/2022
11/11/2022
11/12/2022
11/13/2022
11/14/2022
11/15/2022
11/16/2022
11/17/2022
11/18/2022
11/19/2022
11/20/2022
11/21/2022
11/22/2022
11/23/2022
11/24/2022
11/25/2022
11/26/2022
11/27/2022
11/28/2022
11/29/2022
11/30/2022
12/01/2022
12/02/2022
12/03/2022
12/04/2022
12/05/2022
12/06/2022
12/07/2022
12/08/2022
12/09/2022
12/10/2022
12/11/2022
12/12/2022
12/13/2022
12/14/2022
12/15/2022
12/16/2022
12/17/2022
12/18/2022
12/19/2022
12/20/2022
12/21/2022
12/22/2022
12/23/2022
12/24/2022
12/25/2022
12/26/2022
12/27/2022
12/28/2022
12/29/2022
12/30/2022
12/31/2022
01/01/2023
01/02/2023
01/03/2023
01/04/2023
01/05/2023
01/06/2023
01/07/2023
01/08/2023
01/09/2023
01/10/2023
01/11/2023
01/12/2023
01/13/2023
01/14/2023
01/15/2023
01/16/2023

In [7]:
# ssss = ['01/01/2021', '01/09/2021', '01/13/2021', '01/23/2021', '01/28/2021',
#        '02/03/2021', '02/08/2021', '02/10/2021', '02/11/2021', '02/14/2021', '02/15/2021', '02/18/2021', '02/24/2021',
#        '03/12/2021', '03/21/2021', '03/25/2021',
#        '04/03/2021', '04/04/2021', '04/12/2021', '04/14/2021', '04/16/2021', '04/18/2021']
# for sss in ssss:
#     data_list = [d.strftime('%m/%d/%Y') for d in pd.date_range(start=sss, end=sss)]
#     for date in data_list: 
#         print(date)
#         oppo_player(date, '2020-21', s_list)

01/01/2021
01/09/2021
01/13/2021
01/23/2021
01/28/2021
02/03/2021
02/08/2021
02/10/2021
02/11/2021
02/14/2021
02/15/2021
02/18/2021
02/24/2021
03/12/2021
03/21/2021
03/25/2021
04/03/2021
04/04/2021
04/12/2021
04/14/2021
04/16/2021
04/18/2021


## 2. player block

CSV

In [8]:
import pandas as pd
import os
import pickle

In [9]:
s_list = pickle.load(open( "schedule4.p", "rb" ))
t_list = pickle.load(open( "team.p", "rb" ))
t_list

{1610612738: 'BOS',
 1610612742: 'DAL',
 1610612765: 'DET',
 1610612745: 'HOU',
 1610612754: 'IND',
 1610612748: 'MIA',
 1610612760: 'OKC',
 1610612761: 'TOR',
 1610612737: 'ATL',
 1610612739: 'CLE',
 1610612743: 'DEN',
 1610612744: 'GSW',
 1610612746: 'LAC',
 1610612747: 'LAL',
 1610612763: 'MEM',
 1610612749: 'MIL',
 1610612750: 'MIN',
 1610612740: 'NOP',
 1610612752: 'NYK',
 1610612753: 'ORL',
 1610612755: 'PHI',
 1610612756: 'PHX',
 1610612757: 'POR',
 1610612758: 'SAC',
 1610612762: 'UTA',
 1610612764: 'WAS',
 1610612751: 'BKN',
 1610612759: 'SAS',
 1610612766: 'CHA',
 1610612741: 'CHI'}

In [11]:
df = pd.concat([pd.read_csv(os.path.join('22_23', '10.csv')), pd.read_csv(os.path.join('22_23', '11.csv')), pd.read_csv(os.path.join('22_23', '12.csv')), pd.read_csv(os.path.join('22_23', '01.csv')), pd.read_csv(os.path.join('22_23', '02.csv')), pd.read_csv(os.path.join('22_23', '03.csv')), pd.read_csv(os.path.join('22_23', '04.csv'))])

In [12]:
df

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes
0,Tue Oct 18 2022,7:30p,Philadelphia 76ers,117,Boston Celtics,126,Box Score,NaN,19156.0,TD Garden,NaN
1,Tue Oct 18 2022,10:00p,Los Angeles Lakers,109,Golden State Warriors,123,Box Score,NaN,18064.0,Chase Center,NaN
2,Wed Oct 19 2022,7:00p,Orlando Magic,109,Detroit Pistons,113,Box Score,NaN,20190.0,Little Caesars Arena,NaN
3,Wed Oct 19 2022,7:00p,Washington Wizards,114,Indiana Pacers,107,Box Score,NaN,15027.0,Gainbridge Fieldhouse,NaN
4,Wed Oct 19 2022,7:30p,Houston Rockets,107,Atlanta Hawks,117,Box Score,NaN,17878.0,State Farm Arena,NaN
...,...,...,...,...,...,...,...,...,...,...,...
65,Sun Apr 9 2023,3:30p,Utah Jazz,117,Los Angeles Lakers,128,Box Score,NaN,18997.0,Crypto.com Arena,NaN
66,Sun Apr 9 2023,3:30p,New Orleans Pelicans,108,Minnesota Timberwolves,113,Box Score,NaN,18978.0,Target Center,NaN
67,Sun Apr 9 2023,3:30p,Memphis Grizzlies,100,Oklahoma City Thunder,115,Box Score,NaN,16601.0,Paycom Center,NaN
68,Sun Apr 9 2023,3:30p,Los Angeles Clippers,119,Phoenix Suns,114,Box Score,NaN,17071.0,Footprint Center,NaN


In [13]:
name = ['Boston Celtics', 'Dallas Mavericks', 'Detroit Pistons', 'Houston Rockets', 'Indiana Pacers', 'Miami Heat', 'Oklahoma City Thunder',
       'Toronto Raptors', 'Atlanta Hawks', 'Cleveland Cavaliers', 'Denver Nuggets', 'Golden State Warriors', 'Los Angeles Clippers', 
       'Los Angeles Lakers', 'Memphis Grizzlies', 'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks',
       'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings', 'Utah Jazz', 'Washington Wizards',
       'Brooklyn Nets', 'San Antonio Spurs', 'Charlotte Hornets', 'Chicago Bulls']
new_list = dict(zip(list(t_list.keys()), name))

In [14]:
m_list = dict(zip(['10', '11', '12', '01', '02', '03', '04'], ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr']))
# m_list = dict(zip(['12', '01', '02', '03', '04', '05'], ['Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May']))

In [15]:
HA_dict = s_list.copy()
res = dict.fromkeys(list(s_list.keys()))

In [16]:
res

{'10/18/2022': None,
 '10/19/2022': None,
 '10/20/2022': None,
 '10/21/2022': None,
 '10/22/2022': None,
 '10/23/2022': None,
 '10/24/2022': None,
 '10/25/2022': None,
 '10/26/2022': None,
 '10/27/2022': None,
 '10/28/2022': None,
 '10/29/2022': None,
 '10/30/2022': None,
 '10/31/2022': None,
 '11/01/2022': None,
 '11/02/2022': None,
 '11/03/2022': None,
 '11/04/2022': None,
 '11/05/2022': None,
 '11/06/2022': None,
 '11/07/2022': None,
 '11/08/2022': None,
 '11/09/2022': None,
 '11/10/2022': None,
 '11/11/2022': None,
 '11/12/2022': None,
 '11/13/2022': None,
 '11/14/2022': None,
 '11/15/2022': None,
 '11/16/2022': None,
 '11/17/2022': None,
 '11/18/2022': None,
 '11/19/2022': None,
 '11/20/2022': None,
 '11/21/2022': None,
 '11/22/2022': None,
 '11/23/2022': None,
 '11/24/2022': None,
 '11/25/2022': None,
 '11/26/2022': None,
 '11/27/2022': None,
 '11/28/2022': None,
 '11/29/2022': None,
 '11/30/2022': None,
 '12/01/2022': None,
 '12/02/2022': None,
 '12/03/2022': None,
 '12/04/2022'

In [17]:
for day in HA_dict:
    day_s = m_list[day[:2]]+' '+day[3:5]+' '+day[6:]
    match_res = list()
    for match in HA_dict[day]:
        teams = list(match.keys())
        z = df[df['Date'].str.contains(day_s)]
        f = True
        for index, row in z.iterrows():
            if row['Visitor/Neutral'] == new_list[teams[0]]:
                match_res.append({teams[1]: match[teams[1]], teams[0]: match[teams[0]]})
                f = False
                break
        if f:
            match_res.append(match)
    res[day] = match_res

In [18]:
res

{'10/18/2022': [{1610612738: 1, 1610612755: 0},
  {1610612744: 1, 1610612747: 0}],
 '10/19/2022': [{1610612737: 1, 1610612745: 0},
  {1610612751: 0, 1610612740: 1},
  {1610612759: 0, 1610612766: 1},
  {1610612748: 0, 1610612741: 1},
  {1610612761: 1, 1610612739: 0},
  {1610612756: 1, 1610612742: 0},
  {1610612762: 1, 1610612743: 0},
  {1610612765: 1, 1610612753: 0},
  {1610612754: 0, 1610612764: 1},
  {1610612763: 1, 1610612752: 0},
  {1610612750: 1, 1610612760: 0},
  {1610612758: 0, 1610612757: 1}],
 '10/20/2022': [{1610612747: 0, 1610612746: 1},
  {1610612755: 0, 1610612749: 1}],
 '10/21/2022': [{1610612737: 1, 1610612753: 0},
  {1610612748: 0, 1610612738: 1},
  {1610612751: 1, 1610612761: 0},
  {1610612766: 0, 1610612740: 1},
  {1610612764: 1, 1610612741: 0},
  {1610612744: 0, 1610612743: 1},
  {1610612752: 1, 1610612765: 0},
  {1610612745: 0, 1610612763: 1},
  {1610612754: 0, 1610612759: 1},
  {1610612750: 0, 1610612762: 1},
  {1610612757: 1, 1610612756: 0}],
 '10/22/2022': [{16106

In [19]:
pickle.dump(res, open('home_y_22_23.p', 'wb'))